In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ompy as om
import logging
import pandas as pd
import json

import scipy as sp
#%matplotlib widget
from ipywidgets import widgets #interact, interactive, fixed, interact_manual

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)


In [2]:
om.__full_version__;
np.random.seed(1382398)

In [3]:
def check_kevperbin(energy):
    kev = energy[3]-energy[2]
    return kev*1e3


def getMedianQuantile(func):
    """
    Estimating median and the 
    68% confidence interval
    and the mean
    and the standard error of mean (sem)
    """
    func = pd.DataFrame(func)
    median = pd.DataFrame.median(func)
    error_lower = func.quantile(0.16)
    error_upper = func.quantile(0.84)
    y_top = error_upper-median 
    y_bot = median-error_lower
    error = (y_bot, y_top)
    return median, error_upper, error_lower, error

def read_OMPy(datetime):
    path = 'analysis/OMPy/normalized_data/U233_2021-'+datetime
    
    energy_gsf = pd.read_csv(path+'/data/gsf_0.txt', sep=" ", header=None, skiprows=[0, -1])[0]*1e-3
    energy_nld = pd.read_csv(path+'/data/nld_0.txt', sep=" ", header=None, skiprows=[0, -1])[0]*1e-3
    gsf = []
    nld = []
    ensemble_members = 100
    for i in range(ensemble_members-1):
        gsf.append(pd.read_csv(path+'/data/gsf_%.0i.txt'%i, sep=" ", header=None, skiprows=[0, -1])[1])
        nld.append(pd.read_csv(path+'/data/nld_%.0i.txt'%i, sep=" ", header=None, skiprows=[0, -1])[1])
    
    gsf_median, gsf_error_upper, gsf_error_lower, gsf_error = getMedianQuantile(gsf)
    nld_median, nld_error_upper, nld_error_lower, nld_error = getMedianQuantile(nld)
    
    nld_vals = [energy_nld, nld_median, nld_error_upper, nld_error_lower, nld_error]
    gsf_vals = [energy_gsf, gsf_median, gsf_error_upper, gsf_error_lower, gsf_error]
    
    return gsf_vals, nld_vals

In [4]:
def read_write_OMPy(path):
    """
    Read data from saved OMPy results.
    Results to file
    Calculate median and error of results
    Save results to file
    """
    path = path+'/'
    
    infile = open(path+'README.txt', 'r')
    for line in infile:
        values = line.split()
        if not values:
            continue
        if values[0] == 'keV':
            kev_per_bin_wish = float(values[3])
            continue
        if values[0] == 'Ex_min:':
            Ex_min = float(values[1])
            continue
        if values[0] == 'Ex_max:':
            Ex_max = float(values[1])
            continue
        if values[0] == 'Eg_min:':
            Eg_min = float(values[1])
            continue
        if values[0] == 'Eg_max:':
            Eg_max = float(values[1])
            continue
        if values[0] == 'Gg:':
            Gg = float(values[1])
            Gg_err = float(values[3])
            continue
    infile.close()
    
    parameter_dict = {'keV_per_bin': kev_per_bin_wish, 'Ex_min': Ex_min, 'Ex_max': Ex_max, 'Eg_min': Eg_min, 'Eg_max': Eg_max, 'Gg': Gg, 'Gg_err': Gg_err,}
    dataFrame1 = pd.DataFrame(parameter_dict, index=[0])
    
    energy_gsf = pd.read_csv(path+'data/gsf_0.txt', sep=" ", header=None, skiprows=[0, -1])[0]*1e-3
    energy_nld = pd.read_csv(path+'data/nld_0.txt', sep=" ", header=None, skiprows=[0, -1])[0]*1e-3
    gsf = []
    nld = []
    
    for i in range(100-1):
        gsf.append(pd.read_csv(path+'data/gsf_%.0i.txt'%i, sep=" ", header=None, skiprows=[0, -1])[1])
        nld.append(pd.read_csv(path+'data/nld_%.0i.txt'%i, sep=" ", header=None, skiprows=[0, -1])[1])
    
    gsf_median, gsf_error_upper, gsf_error_lower, gsf_error = getMedianQuantile(gsf)
    nld_median, nld_error_upper, nld_error_lower, nld_error = getMedianQuantile(nld)
    
    OMPy_dict = {'E_gamma': energy_gsf, 'gSF-median':gsf_median, 'gSF-error': gsf_error, 'E_x': energy_nld, 'NLD-median': nld_median, 'NLD-error': nld_error}
    
    #write to file
    dataFrame2 = pd.DataFrame(OMPy_dict)
    dataFrame = pd.concat([dataFrame1, dataFrame2])
    dataFrame.to_csv(path+'data_for_analysis.csv')



In [5]:
#Declaring fit functions

def SLo(E_gamma, E, Gamma, sigma, T_f):
    """
    Standard Lorentzian
    E_gamma = The gamma-energy on the x-axis
    Gamma = 
    E =
    sigma =
    """
    factor = 8.674e-8
    denominator = (E_gamma**2 - E**2)**2 + (Gamma**2 * E_gamma**2)
    
    return factor * sigma * Gamma**2 * E_gamma / denominator

def Gamma_k(E_gamma, Gamma, E, T_f):
    """
    E_gamma = The gamma-energy on the x-axis
    Gamma = 
    E =
    sigma =
    """
    return Gamma * (E_gamma**2 + (2*np.pi*T_f)**2) / (E**2)

def GLo(E_gamma, E, Gamma, sigma, T_f):
    """
    Generalized Lorentzian
    E_gamma = The gamma-energy on the x-axis
    Gamma = 
    E =
    sigma = 
    T_f = 
    """
    factor = 8.674e-8
    A = E_gamma * Gamma_k(E_gamma, Gamma, E, T_f) / ( (E_gamma**2 - E**2)**2 + (E_gamma * Gamma_k(E_gamma, Gamma, E, T_f))**2 )
    B = 0.7 * Gamma_k(E_gamma, Gamma, E, T_f) / (E**3)
    
    GLo = factor * sigma * Gamma * (A + B)
    return GLo

def FitFunctionStrength(x, par):
    """
    All different resonances and parameters of the fit to the experimental strength function
    GLo1 and GLo2 parameters are for the two peaks of the GDR
    pyg1 and pyg2 are for the two peaks of the pygme resonances
    SR are for the Scissor Resonance
    
    E_gamma = The gamma-energy on the x-axis
    Gamma_r_* = 
    E_r_* = 
    sigma_r_* = 
    T_f = Constant temperature for the GLo fits.
    
    """
    E_gamma = x[0]
    
    Gamma_r_GLo1 = par[0]
    E_r_GLo1 = par[1]
    sigma_r_GLo1 = par[2]
    
    Gamma_r_GLo2 = par[3]
    E_r_GLo2 = par[4]
    sigma_r_GLo2 = par[5]
    
    T_f = par[6]
    
    Gamma_pyg1 = par[7]
    E_pyg1 = par[8]
    sigma_pyg1 = par[9]
    
    Gamma_pyg2 = par[10]
    E_pyg2 = par[11]
    sigma_pyg2 = par[12]
    
    Gamma_SR = par[13]
    E_SR = par[14]
    sigma_SR = par[15]
    
    GLo1 = GLo(E_gamma, Gamma_r_GLo1, E_r_GLo1, sigma_r_GLo1, T_f)
    GLo2 = GLo(E_gamma, Gamma_r_GLo2, E_r_GLo2, sigma_r_GLo2, T_f)
    SLo_pyg1 = SLo(E_gamma, Gamma_pyg1, E_pyg1, sigma_pyg1)
    SLo_pyg2 = SLo(E_gamma, Gamma_pyg2, E_pyg2, sigma_pyg2)
    SLo_SR = SLo(E_gamma, Gamma_SR, E_SR, sigma_SR)
    
    return GLo1 + GLo2 + SLo_pyg1 + SLo_pyg2 + SLo_SR

def FitFunctionE1(x, par):
    E_gamma = x[0]
    Gamma_r_GLo1 = par[0]
    E_r_GLo1 = par[1]
    sigma_r_GLo1 = par[2]
    Gamma_r_GLo2 = par[3]
    E_r_GLo2 = par[4]
    sigma_r_GLo2 = par[5]
    T_f = par[6]
    return GLo(E_gamma, Gamma_r_GLo1, E_r_GLo1, sigma_r_GLo1, T_f) + GLo(E_gamma, Gamma_r_GLo2, E_r_GLo2, sigma_r_GLo2, T_f)

def FitPygmy(x, par):
    E_gamma = x[0]
    Gamma = par[0]
    E = par[1]
    sigma = par[2]
    return SLo(E_gamma, Gamma, E, sigma)

In [6]:
def cs_to_gsf(Eg, data):
    # input in MeV and milibarn -> use this conversion factor.
    exp = 8.6737e-8 # DOBBELTSJEKK TALLET! # pi**2 * hbar**2 * c**2
    return data*exp/Eg

def extract_JSON(data, x_i,y_i,yerror_i):
    """ Extracting data from input files from MAMA, adding them to useful arrays."""
    x = []; y = [] ; yerror = []

    for j in range(len(data)):
        y.append(data[j][y_i]) # MeV
        yerror.append(data[j][yerror_i]) # MeV
        x.append(data[j][x_i])

    x = np.array(x) ; y = np.array(y) ; yerror = np.array(yerror)
    return x,y,yerror